# Count number of motion outliers

In [1]:
%matplotlib inline
from __future__ import division
import re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from glob import glob
from os.path import join, basename, splitext
import pandas as pd

In [2]:
# Directions per http://web.mit.edu/fsl_v5.0.8/fsl/doc/wiki/POSSUM(2f)UserGuide.html
rot_dirs = ['Pitch', 'Roll', 'Yaw']
trans_dirs = ['X', 'Y', 'Z']
colors = sns.color_palette('husl')

data_dir = '/scratch/PSB6351_2017/students/salo/data/preproc/'
out_dir = '/scratch/PSB6351_2017/students/salo/week13/outliers/'

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = [12, 8]

In [3]:
# Get ylims
subjects = [f for f in glob(join(data_dir, 'sub-*'))]
r_lim = 0.
t_lim = 0.
for s in subjects:
    motpar_files = sorted([f for f in glob(join(s, 'preproc/motion/*.par'))])
    for f in motpar_files:
        dat = np.loadtxt(f)
        t, r = dat[:, :3], dat[:, 3:]
        r_lim = np.max([r_lim, np.max(np.abs(r))])
        t_lim = np.max([t_lim, np.max(np.abs(t))])
r_lim = np.ceil(r_lim * 100.) / 100.
t_lim = np.ceil(t_lim * 10.) / 10.

In [4]:
def framewise_displacement(vals):
    mot_pars = np.copy(vals)
    mot_pars[:, 3:] = mot_pars[:, 3:] * 50  # Convert radians to mm
    mot_pars = np.vstack((np.array([[0, 0, 0, 0, 0, 0]]),
                          np.diff(mot_pars, axis=0)))
    fd = np.sum(np.abs(mot_pars), axis=1)
    return fd

In [5]:
thresholds = [0.5, 1.0]
thresh_names = ['0.5', '1.0']
subjects = sorted([basename(f) for f in glob(join(data_dir, 'sub-*'))])
rows = []
for s in subjects:
    motpar_files = sorted([basename(f) for f in glob(join(data_dir, s, 'preproc/motion/*.par'))])
    tasks = sorted(list(set([re.search('task-(.*)_run', f).group(1) for f in motpar_files])))
    for t in tasks:
        rel_files = sorted([f for f in motpar_files if 'task-{0}_run'.format(t) in f])
        n_runs = len(rel_files)
        for run in range(n_runs):
            run_name = re.search('(run-[0-9][0-9])', rel_files[run]).group(0)
            dat = np.loadtxt(join(data_dir, s, 'preproc/motion/', rel_files[run]))
            fd = framewise_displacement(dat)
            prop_bad = [0 for _ in range(len(thresholds))]
            for i in range(len(thresholds)):
                cens_vols = np.where(fd>thresholds[i])[0]
                bef_cens_vols = cens_vols - 1
                aft_cens_vols = cens_vols + 1
                aft2_cens_vols = cens_vols + 2
                all_vols = np.array(range(len(fd)))
                
                # Combine bad vols, vols before, and vols after
                all_cens_vols = np.union1d(bef_cens_vols, cens_vols)
                all_cens_vols = np.union1d(all_cens_vols, aft_cens_vols)
                all_cens_vols = np.union1d(all_cens_vols, aft2_cens_vols)
                
                # Remove censored index outside range
                red_cens_vols = np.intersect1d(all_vols, all_cens_vols)
                np.savetxt(join(out_dir, 'FD{0}/{1}-{2}-{3}.txt'.format(thresh_names[i],
                                                                        s, t, run_name)),
                           red_cens_vols, fmt='%i')
                prop_bad[i] = len(red_cens_vols) / len(all_vols)
            rows += [[s, t, run] + prop_bad]
df = pd.DataFrame(data=rows, columns=['Subject', 'Task', 'Run']+['FD>'+th for th in thresh_names])
df.to_csv('outliers/hw.csv', index=False)